# import

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome import service as fs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from collections import Counter
import time, datetime, MeCab, random, os
import pandas as pd

white = '/home/ichikawa17/01_yahooScraping/'

# chromeDriver

In [9]:
#chromeをheadlessで起動
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_service = fs.Service(executable_path="/usr/local/bin/chromedriver/chromedriver95")
driver = webdriver.Chrome(service=chrome_service,  options=chrome_options)
#driver = webdriver.Chrome("/usr/local/bin/chromedriver")

# ニュースリストの取得

In [10]:
top_page = "https://news.yahoo.co.jp/ranking/comment/"
page = '?page='
older = '&order=older'
driver.get(top_page)
ol = driver.find_element(By.CLASS_NAME,'newsFeed_list')
li = ol.find_elements(By.CLASS_NAME,'newsFeed_item_link')
links = [i.get_attribute('href') for i in li]
for link in links:
    CommentList = []
    driver.get(link)
    title = driver.title
    comment_link = driver.find_element(By.CLASS_NAME,'news-comment-plugin').get_attribute('data-full-page-url')
    
    driver.get(comment_link)
    comment_link = driver.current_url
    
    driver.get(comment_link+page+'1'+older)
    iframe = driver.find_element(By.CLASS_NAME,'news-comment-plguin-iframe')
    driver.switch_to.frame(iframe)
    
    num_of_comment = driver.find_element(By.CLASS_NAME,'counter').text
    driver.switch_to.default_content()
    end_page = int(int(num_of_comment.split('/')[-1].strip('件'))/10)+1
    print(end_page)
    for num_of_comment in range(1,end_page):
        driver.get(comment_link+
                   page+
                   str(num_of_comment)+
                   older)
        print(driver.current_url)
        iframe = driver.find_element(By.CLASS_NAME,'news-comment-plguin-iframe')
        driver.switch_to.frame(iframe)
        try:
            comment_list = driver.find_element(By.ID,'comment-list-item')
            comments = comment_list.find_elements(By.CLASS_NAME,'commentListItem')
            for comment in comments:
            
                CommentList.append(comment.find_element(By.CLASS_NAME,'cmtBody').text)
        except:
            pass
    df = pd.Series(data=CommentList)
    df.to_csv('./data_set/'+title+'.csv')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".news-comment-plugin"}
  (Session info: headless chrome=96.0.4664.110)
Stacktrace:
#0 0x563895abfac3 <unknown>
#1 0x5638955998f8 <unknown>
#2 0x5638955cf6fa <unknown>
#3 0x563895602004 <unknown>
#4 0x5638955ecb2d <unknown>
#5 0x5638955ffca1 <unknown>
#6 0x5638955ec9f3 <unknown>
#7 0x5638955c2e14 <unknown>
#8 0x5638955c3e05 <unknown>
#9 0x563895af125e <unknown>
#10 0x563895b06afa <unknown>
#11 0x563895af21b5 <unknown>
#12 0x563895b084c8 <unknown>
#13 0x563895ae695b <unknown>
#14 0x563895b23298 <unknown>
#15 0x563895b23418 <unknown>
#16 0x563895b3ebed <unknown>
#17 0x7ff114147609 <unknown>


In [ ]:
link = links[2]
CommentList = []
driver.get(link)
title = driver.title
print(title)
comment_link = driver.find_element(By.CLASS_NAME,'news-comment-plugin').get_attribute('data-full-page-url')

driver.get(comment_link)
comment_link = driver.current_url

driver.get(comment_link+page+'1'+older)
iframe = driver.find_element(By.CLASS_NAME,'news-comment-plguin-iframe')
driver.switch_to.frame(iframe)

num_of_comment = driver.find_element(By.CLASS_NAME,'counter').text
driver.switch_to.default_content()
end_page = int(int(num_of_comment.split('/')[-1].strip('件'))/10)+1
print(end_page)
for num_of_comment in range(1,end_page):
    driver.get(comment_link+page+'1'+older)
    iframe = driver.find_element(By.CLASS_NAME,'news-comment-plguin-iframe')
    driver.switch_to.frame(iframe)

    comment_list = driver.find_element(By.ID,'comment-list-item')
    comments = comment_list.find_elements(By.CLASS_NAME,'commentListItem')
    for comment in comments:
        CommentList.append(comment.find_element(By.CLASS_NAME,'cmtBody').text)
        df = pd.Series(data=CommentList)
        df.to_csv(title+'.csv')

# コメント情報の取得

In [ ]:
df_NC = pd.read_csv(white+'/01_NewsList/'+str(dt_NL)+'.csv')
url = df_NC['comment_url'][0]
title = df_NC['news_title'][0]
driver.get(url)
print(title)

#コメント情報の取得

comment_boxes = []

#df = pd.DataFrame(columns=['number','日付','ユーザー名','コメント','good','bad'])
df_NC = pd.DataFrame(columns=['number','ユーザー名','コメント'])
start = 1
end = 200
base_url = driver.current_url
page_url = '?page='
order_url = '&order=older'

iframe = driver.find_element(By.CLASS_NAME,"news-comment-plguin-iframe")
# driver.switch_to_frame(iframe)
driver.switch_to.frame(iframe)
end = driver.find_element(By.CLASS_NAME,"pagenation")
pages = end.find_elements(By.CLASS_NAME,"rapidnofollow")
last_page = pages[7].text
driver.switch_to.default_content()
print(last_page)
for page in range(start,int(last_page)+1):

    driver.get(base_url+page_url+str(page)+order_url) 
    print(driver.current_url)
#    time.sleep(1)
    iframe = driver.find_element(By.CLASS_NAME,"news-comment-plguin-iframe")
    driver.switch_to.frame(iframe)

    comment_boxes = driver.find_elements(By.CLASS_NAME,"root")
    page_comment =0

    for comment_box in comment_boxes:



        page_comment +=1
        comment_number = str(page)+'-'+str(page_comment)

        #ユーザー名取得
        elem_name = comment_box.find_element(By.CLASS_NAME,"rapidnofollow")
        name = elem_name.text

        #コメント取得
        try:
            elem_comment = comment_box.find_element(By.CLASS_NAME,"cmtBody")
        except:
            elem_comment = comment_box.find_element(By.CLASS_NAME,"yjxComment")

        comment = elem_comment.text
        comment.strip("\n")

        #good数取得
        try:
            agree_box = comment_box.find_element(By.CLASS_NAME,"good")
            elem_agree = agree_box.find_element(By.CLASS_NAME,"userNum")
            agree = elem_agree.text
        except:
            pass

        #bad数取得
        try:
            disagree_box = comment_box.find_element(By.CLASS_NAME,"bad")
            elem_disagree = disagree_box.find_element(By.CLASS_NAME,"userNum")
            disagree = elem_disagree.text
        except:
            pass
        #df.loc[(page-1)*10+page_comment] = [comment_number,date, name, comment, agree, disagree]
        df_NC.loc[(page-1)*10+page_comment] = [comment_number, name, comment]


df_NC.to_csv(white+'/02_NewsComment/'+dt_NC+title+".csv")
print("csv is ok")
driver.quit()

In [ ]:
# df_NL = pd.DataFrame(columns=['news_title','comment_url'])

# #ニュース記事への移動
# for a in range(8):
#     section = driver.find_element(By.TAG_NAME,"section")
#     news_list = section.find_elements(By.TAG_NAME,"li")
#     mv_news = news_list[a].click()
#     time.sleep(0.1)
#     news_main = driver.find_element(By.TAG_NAME,"article")
#     news_title = news_main.find_elements_by_tag_name("p")
#     print(news_title[2].text)

# #コメントページへの移動
#     try:
#         news_comment_url = driver.find_element(By.CLASS_NAME,"news-comment-plugin").get_attribute("data-full-page-url")
#         df_NL.loc[a] = [news_title[2].text,news_comment_url]    
#     except:
#         df_NL.loc[a] = [news_title[2],'This news page have no comment.']    
#     finally:
#         driver.get(topPage) 

# dt_now = datetime.datetime.now()
# dt_NL = dt_now.strftime('%m%d_%H%M')
# dt_NC = dt_now.strftime('%m%d%H%M')
# df_NL.to_csv(white+'01_NewsList/'+dt_NL+'.csv')
# print("csv is ok")